## Sensitive analysis:


In [1]:
from sast.sast import *
from sktime.datasets import load_UCR_UEA_dataset, tsc_dataset_names
from sktime.classification.kernel_based import RocketClassifier
import time
import pandas as pd


/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/mass_ts/_mass_ts.py:17: UserWarning: GPU support will not work. You must pip install mass-ts[gpu].
  warnings.warn(


### Select Datasets for hypertunning RSAST

It is runned RSAST in a set of UCR datasets with a predefined number of runs ("runs"). Then, it is selected a range ("range_total") between 1, 10, 100 and 10000 for the selected dataset.

In [15]:
ds_sens = tsc_dataset_names.univariate_equal_length
#ds_sens = ['ACSF1', 'Adiac', 'ArrowHead', 'Beef', 'BeetleFly', 'BirdChicken', 'BME', 'Car', 'CBF', 'Chinatown', 'ChlorineConcentration', 'CinCECGTorso', 'Coffee']
ds_sens = ['Chinatown','BirdChicken','Coffee' ]
max_ds=len(ds_sens) #exploring dataset in UEA & UCR Time Series Classification Repository
#print(ds_sens)
print(ds_sens)


['Chinatown', 'BirdChicken', 'Coffee']


In [25]:
#ds='Chinatown' # Chosing a dataset from # Number of classes to consider
runs = 1
range_total = [1,10,100,1000]
#list
df_result = {}
list_score = []
list_time = []
list_dataset = []
list_hyperparameter = []
list_method = []
list_rpoint = []
list_nb_per_class = []

for ds in ds_sens:
    print("ds="+ds)
    X_train, y_train = load_UCR_UEA_dataset(name=ds, extract_path='data', split="train", return_type="numpy2d")
    X_test, y_test = load_UCR_UEA_dataset(name=ds, extract_path='data', split="test", return_type="numpy2d")
    for i in range(runs):
        """
        start = time.time()
        rocket= RocketClassifier(num_kernels=10_000)
        rocket.fit(X_train,y_train)
        end = time.time()
        list_score.append(rocket.score(X_test,y_test))
        list_time.append(end-start)
        list_dataset.append(ds)
        list_hyperparameter.append("num_kernels=10_000")
        list_rpoint.append(str(0))
        list_nb_per_class.append(str(0))
        list_method.append("Rocket")
        """
        for p in range_total:
            for g in range_total:
                if p ==0:
                    p=1
                if g ==0:
                    g=1
                start = time.time()
                random_state = None
                rsast_ridge = RSAST(n_random_points=p,nb_inst_per_class=g, classifier=RidgeClassifierCV())
                rsast_ridge.fit(X_train, y_train)
                end = time.time()
                list_score.append(rsast_ridge.score(X_test,y_test))
                list_time.append(end-start)
                list_dataset.append(ds)
                list_hyperparameter.append("n_random_points="+str(p)+" nb_inst_per_class="+str(g))
                list_rpoint.append(str(p))
                list_nb_per_class.append(str(g))
                list_method.append("Rsast")
                print("n_random_points="+str(p)+" nb_inst_per_class="+str(g))

df_result['accuracy']=list_score
df_result['time']=list_time
df_result['dataset_name']=list_dataset
df_result['classifier_name']=list_hyperparameter
df_result['rpoint']=list_rpoint
df_result['nb_per_class']=list_nb_per_class
df_result['method']=list_method

ds=Chinatown
n_random_points=1 nb_inst_per_class=1
n_random_points=1 nb_inst_per_class=10
n_random_points=1 nb_inst_per_class=100
n_random_points=1 nb_inst_per_class=1000
n_random_points=10 nb_inst_per_class=1
n_random_points=10 nb_inst_per_class=10
n_random_points=10 nb_inst_per_class=100
n_random_points=10 nb_inst_per_class=1000
n_random_points=100 nb_inst_per_class=1
n_random_points=100 nb_inst_per_class=10
n_random_points=100 nb_inst_per_class=100
n_random_points=100 nb_inst_per_class=1000
n_random_points=1000 nb_inst_per_class=1
n_random_points=1000 nb_inst_per_class=10
n_random_points=1000 nb_inst_per_class=100
n_random_points=1000 nb_inst_per_class=1000
ds=BirdChicken
n_random_points=1 nb_inst_per_class=1
n_random_points=1 nb_inst_per_class=10
n_random_points=1 nb_inst_per_class=100
n_random_points=1 nb_inst_per_class=1000
n_random_points=10 nb_inst_per_class=1
n_random_points=10 nb_inst_per_class=10
n_random_points=10 nb_inst_per_class=100
n_random_points=10 nb_inst_per_class=1

In [32]:
df_result=pd.DataFrame(df_result)
pd.pivot_table(df_result, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,100,1000
rpoint,,,,
1,0.893343,0.929592,0.921817,0.929592
10,0.907920,0.939310,0.939310,0.943197
100,0.901118,0.945141,0.944169,0.944169
1000,0.909038,0.945141,0.945141,0.944169


In [3]:
# create a pivot table
df_result.to_csv("df_overall_results.csv") 
pivot_table = pd.pivot_table(df_result, values='accuracy', index=['classifier_name','dataset_name'], aggfunc='mean')
pivot_table.to_csv("df_pivot_tunning_hyper.csv") 
#pivot_table



AttributeError: 'dict' object has no attribute 'to_csv'